
# HDX Tabular Data Endpoints: Jupyter Notebook

This notebook shows how to query the HDX Tabular Data endpoints from Python using both native (`datastore_search`) and SQL (`datastore_search_sql`) with filters and joins along with investigating the schema through the `datastore_info` endpoint. It also includes an advanced section on how to use the HDX metadata endpoints to search for datasets.

**Please note:**  
- Use your own API token. Do not commit or share it. Treat it like a password and keep it safe.
- If using Google Colab, you can set your token at runtime in a cell. Do not save it to Drive or source control.  
- Replace placeholder values like `"RESOURCE_ID"` with real ones.
- Use the [documentation](https://un-ocha-centre-for-humanitarian.gitbook.io/hdx-docs/build-with-hdx/build-with-hdx/hdx-api/tabular-data-endpoints) to answer any other questions!


## 1. Set your API token

Set your API token by saving it as an environment variable (`API_TOKEN`). See instructions on how to generate your token in the [documentation](https://un-ocha-centre-for-humanitarian.gitbook.io/hdx-docs/build-with-hdx/build-with-hdx/overview/hdx-core-concepts). In the notebook, load it with a config file or create a secret in notebook editors like Google Collab. Do not hardcode or commit your token in the notebook.


In [1]:
import os

# Retrieve the HDX API token from environment variables
API_TOKEN = os.getenv("HDX_API_TOKEN")

# Check that the token exists
if not API_TOKEN:
    raise ValueError("HDX_API_TOKEN not found in environment variables")

print("HDX API_TOKEN loaded successfully.")

HDX API_TOKEN loaded successfully.


## 2. Set variables: Base URL, resource IDs, and headers
You will need to set the base for the API endpoints and `resource_id` of the datastore active HDX resources. This can be found via the UI, exact instructions in the [documentation](https://un-ocha-centre-for-humanitarian.gitbook.io/hdx-docs/build-with-hdx/build-with-hdx/hdx-api/tabular-data-endpoints/available-data-and-resource-ids). (See advanced section below on how to query the HDX metadata endpoints for `resource_id` as well.)

In [2]:
BASE = "https://data.humdata.org/api/action"
HNO_RESOURCE_ID = "45036735-305b-42ae-9aef-b941d6dcb6d6" # Replace with your resource id, this is for affected people
RAINFALL_RESOURCE_ID = "0e6fe8ce-f7c5-4320-bc57-bc95ed7fcd7b"  # Replace with your resource id, this is for rainfall
HEADERS = {"Authorization": API_TOKEN} if API_TOKEN else {}
print("Using resources:", HNO_RESOURCE_ID, "and", RAINFALL_RESOURCE_ID)

Using resources: 45036735-305b-42ae-9aef-b941d6dcb6d6 and 0e6fe8ce-f7c5-4320-bc57-bc95ed7fcd7b


## 3. Load packages and helper functions
Load the required packages and basic backoff, native calls, SQL calls, and paginated fetch helpers. You may not use all of these!

In [3]:
import json
import time
import urllib.parse
import requests
import pandas as pd

# GET with basic exponential backoff and JSON response
def get_with_backoff(url, headers=None, tries=5, timeout=60):
    for i in range(tries):
        r = requests.get(url, headers=headers or {}, timeout=timeout)
        if r.status_code in (429, 500, 502, 503, 504):
            # back off and try again
            time.sleep(2 ** i)
            continue
        r.raise_for_status()
        return r.json()
    raise RuntimeError(f"Request failed after {tries} tries: {url}")

# Search datastore with pagination limit of 32,000
def datastore_search(resource_id, limit=32000, offset=0, filters=None, fields=None):
    params = {
        "resource_id": resource_id,
        "limit": str(limit),
        "offset": str(offset),
    }
    if filters:
        params["filters"] = json.dumps(filters)
    if fields:
        params["fields"] = ",".join(fields)
    url = f"{BASE}/datastore_search?{urllib.parse.urlencode(params)}"
    return get_with_backoff(url, headers=HEADERS)

# Native fetch with pagination using limit and offset
def fetch_all_native(resource_id, page_size=32000, filters=None, fields=None):
    out = []
    offset = 0
    total = None
    while True:
        resp = datastore_search(resource_id, limit=page_size, offset=offset, filters=filters, fields=fields)
        if not resp.get("success"):
            raise RuntimeError(f"API indicated failure: {resp}")
        result = resp["result"]
        if total is None:
            total = result.get("total", 0)
        rows = result.get("records", [])
        if not rows:
            break
        out.extend(rows)
        offset += len(rows)
        if offset >= total:
            break
    return pd.DataFrame(out)

# SQL search
def datastore_search_sql(sql):
    q = {"sql": sql}
    url = f"{BASE}/datastore_search_sql?{urllib.parse.urlencode(q)}"
    return get_with_backoff(url, headers=HEADERS)

# Iterate using SQL with stable ORDER BY and OFFSET
def fetch_all_sql(resource_id, order_by, page_size=32000, where=None, fields="*"):
    all_rows = []
    offset = 0

    while True:
        sql = f'SELECT {fields} FROM "{resource_id}"'
        if where:
            sql += f" WHERE {where}"
        sql += f' ORDER BY "{order_by}" LIMIT {page_size} OFFSET {offset}'

        data = datastore_search_sql(sql)
        rows = data.get("result", {}).get("records", [])
        if not rows:
            break

        all_rows.extend(rows)
        offset += len(rows)
        if len(rows) < page_size:
            break

    return pd.DataFrame.from_records(all_rows) if all_rows else pd.DataFrame()

# Show schema, field types, and index info
def show_schema(resource_id, label):
    url = f"{BASE}/datastore_info?{urllib.parse.urlencode({'id': resource_id})}"
    r = requests.get(url, headers=HEADERS)
    r.raise_for_status()
    data = r.json()

    if not data.get("success"):
        raise RuntimeError(f"Probe failed: {data}")

    result = data.get("result", {}) or {}
    fields = result.get("fields", []) or []
    primary_key = set(result.get("primary_key", []) or [])
    indexes = result.get("indexes", []) or []
    
    indexed_cols = set()
    for idx in indexes:
        if isinstance(idx, (list, tuple)):
            indexed_cols.update(idx)
        elif isinstance(idx, str):
            indexed_cols.add(idx)

    print(f"{label} schema ({len(fields)} fields):")
    for f in fields:
        fid = f.get("id")
        ftype = f.get("type")
        is_index = (fid in primary_key) or (fid in indexed_cols)
        print(f"  {fid}: {ftype} | is_index={bool(is_index)}")

## 4. Advanced: Search with the HDX CKAN API for datastore resources
This is an advanced and optional section showing you how to find `datastore_active` resources within the CKAN API, a separate HDX API endpoint. This is useful for browsing the data catalogue and searching for the data you want. The `resource_id` is what is then used in the datastore API call (native and SQL).

The `resource_id` can also be found in the UI directly as in the earlier example. See more information in the [documentation](https://un-ocha-centre-for-humanitarian.gitbook.io/hdx-docs/build-with-hdx/build-with-hdx/overview/hdx-core-concepts).

In [4]:
# Search CKAN for datasets where title includes "rainfall", country (group) is AFG, and organization is WFP
params = {
    "q": 'title:rainfall',
    "fq": 'organization:wfp +groups:afg',
    "rows": 1000
}

resp = requests.get(f"{BASE}/package_search", params=params, headers=HEADERS)
resp.raise_for_status()
data = resp.json()["result"]
print("Total matching datasets:", data["count"])
datasets = data["results"]

rows = []
for ds in datasets:
    for res in ds.get("resources", []):
        if res.get("datastore_active"):
            rows.append({
                "dataset_name": ds.get("name"),
                "dataset_title": ds.get("title"),
                "organization": (ds.get("organization") or {}).get("name"),
                "resource_name": res.get("name"),
                "resource_id": res.get("id"),
                "url": res.get("url")
            })

ckan_search_df = pd.DataFrame(rows)
print(f"{len(ckan_search_df)} Tabular API active WFP resources found containing 'rainfall' and groups: afg")
display(ckan_search_df.head())

Total matching datasets: 1
2 Tabular API active WFP resources found containing 'rainfall' and groups: afg


,dataset_name,dataset_title,organization,resource_name,resource_id,url
0,afg-rainfall-subnational,Afghanistan: Rainfall Indicators at Subnationa...,wfp,afg-rainfall-subnat-full.csv,0e6fe8ce-f7c5-4320-bc57-bc95ed7fcd7b,https://data.humdata.org/dataset/3b5e8a5c-e4e0...
1,afg-rainfall-subnational,Afghanistan: Rainfall Indicators at Subnationa...,wfp,afg-rainfall-subnat-5ytd.csv,e6c0b120-6e2e-43f2-bf18-ef51f5fa9c20,https://data.humdata.org/dataset/3b5e8a5c-e4e0...



## 5. Minimal probe
Confirm access and the presence of rows using a small native HDX Datastore API call. This will simply confirm that you are able to access the API with your API token and `resource_id`.

*Refer to the [documentation](https://un-ocha-centre-for-humanitarian.gitbook.io/hdx-docs/build-with-hdx/build-with-hdx/hdx-api/tabular-data-endpoints/troubleshooting-and-error-handling) with troubleshooting if you are unable to access with this minimal probe.*


In [5]:
# Minimal probe to confirm datastore access
try:
    probe = datastore_search(RAINFALL_RESOURCE_ID, limit=1)
    if not probe.get("success"):
        raise RuntimeError(f"Probe failed: {probe}")

    result = probe.get("result", {})
    total = result.get("total")
    records = result.get("records", [])

    print("Probe success:", probe.get("success"))
    print("Total rows in table:", total)
    if records:
        print("Example record:")
        print(records[0])
    else:
        print("No rows returned. Try removing filters or checking the resource_id.")

except Exception as e:
    print("Probe request failed:", e)

Probe success: True
Total rows in table: 700161
Example record:
{'_id': 1, 'date': '1981-01-01T00:00:00', 'adm_level': 1, 'adm_id': 900486, 'PCODE': 'AF21', 'n_pixels': 675.0, 'rfh': 11.9792595, 'rfh_avg': 12.415753, 'r1h': None, 'r1h_avg': None, 'r3h': None, 'r3h_avg': None, 'rfq': 97.49369, 'r1q': None, 'r3q': None, 'version': 'final'}


## 6. Information call to see data schema
A simple API call using the `datastore_info` endpoint of the `resource_id` you are calling. This helps us understand the data schema of the resource for use in filtering and joining.

In [6]:
# Confirm datastore_info access and show schema
try:
    show_schema(RAINFALL_RESOURCE_ID, "Rainfall")

except Exception as e:
    print("Probe request failed:", e)

Rainfall schema (15 fields):
  date: timestamp | is_index=False
  adm_level: numeric | is_index=False
  adm_id: numeric | is_index=False
  PCODE: text | is_index=False
  n_pixels: numeric | is_index=False
  rfh: numeric | is_index=False
  rfh_avg: numeric | is_index=False
  r1h: numeric | is_index=False
  r1h_avg: numeric | is_index=False
  r3h: numeric | is_index=False
  r3h_avg: numeric | is_index=False
  rfq: numeric | is_index=False
  r1q: numeric | is_index=False
  r3q: numeric | is_index=False
  version: text | is_index=False


## 7. Native fetch with filters and pagination
Now we can adjust `filters` and `fields` to match the schema of the `resource_id` you are calling. This is just a simple example which can be edited with your desired filters based on the data resource schema and data types that you are calling.

In [7]:
# Example: native fetch with filters and pagination
filters = {
    "PCODE": "AF16" # Replace with a valid column:value in your dataset
}

fields = None

try:
    df_native = fetch_all_native(
        RAINFALL_RESOURCE_ID,
        page_size=32000,
        filters=filters,
        fields=fields
    )

    n_rows = len(df_native)
    print(f"Native fetch returned {n_rows} rows")
    if n_rows == 0:
        print("No rows matched the filter. Try adjusting filters or check column names/types.")
    else:
        display(df_native.head())

except Exception as e:
    print("Native fetch failed:", e)

Native fetch returned 1617 rows


,_id,date,adm_level,adm_id,PCODE,n_pixels,rfh,rfh_avg,r1h,r1h_avg,r3h,r3h_avg,rfq,r1q,r3q,version
0,32341,1981-01-01T00:00:00,1,900506,AF16,357.0,6.285714,10.568534,NaN,NaN,NaN,NaN,72.490540,NaN,NaN,final
1,32342,1981-01-11T00:00:00,1,900506,AF16,357.0,6.319328,9.605509,NaN,NaN,NaN,NaN,77.500404,NaN,NaN,final
2,32343,1981-01-21T00:00:00,1,900506,AF16,357.0,17.761906,13.974603,30.366947,34.148647,NaN,NaN,119.959854,90.34015,NaN,final
3,32344,1981-02-01T00:00:00,1,900506,AF16,357.0,15.184874,27.983007,39.266106,51.563118,NaN,NaN,61.197792,78.25967,NaN,final
4,32345,1981-02-11T00:00:00,1,900506,AF16,357.0,34.843136,22.349020,67.789920,64.306630,NaN,NaN,145.683960,105.02590,NaN,final



## 8. SQL fetch with stable ordering
Here is a SQL example with a deterministic column for `ORDER BY` which can be used with a primary key like `"date"` or `"id"`.


In [8]:
# Example: SQL fetch with stable ordering and pagination
sql_where = "\"PCODE\" = 'AF16'"

try:
    df_sql = fetch_all_sql(
        RAINFALL_RESOURCE_ID,
        order_by="_id",
        page_size=32000,
        where=sql_where,
        fields="*"
    )

    print(f"SQL fetch returned {len(df_sql)} rows")
    display(df_sql.head())

except Exception as e:
    print("SQL fetch failed:", e)

SQL fetch returned 1617 rows


,_id,_full_text,date,adm_level,adm_id,PCODE,n_pixels,rfh,rfh_avg,r1h,r1h_avg,r3h,r3h_avg,rfq,r1q,r3q,version
0,32341,"'-01':2,3 '00':5,6 '1':7 '10.568534':12 '1981'...",1981-01-01T00:00:00,1,900506,AF16,357.0,6.285714,10.568534,NaN,NaN,NaN,NaN,72.490540,NaN,NaN,final
1,32342,"'-01':2 '-11':3 '00':5,6 '1':7 '1981':1 '357.0...",1981-01-11T00:00:00,1,900506,AF16,357.0,6.319328,9.605509,NaN,NaN,NaN,NaN,77.500404,NaN,NaN,final
2,32343,"'-01':2 '-21':3 '00':5,6 '1':7 '119.959854':15...",1981-01-21T00:00:00,1,900506,AF16,357.0,17.761906,13.974603,30.366947,34.148647,NaN,NaN,119.959854,90.34015,NaN,final
3,32344,"'-01':3 '-02':2 '00':5,6 '1':7 '15.184874':11 ...",1981-02-01T00:00:00,1,900506,AF16,357.0,15.184874,27.983007,39.266106,51.563118,NaN,NaN,61.197792,78.25967,NaN,final
4,32345,"'-02':2 '-11':3 '00':5,6 '1':7 '105.0259':16 '...",1981-02-11T00:00:00,1,900506,AF16,357.0,34.843136,22.349020,67.789920,64.306630,NaN,NaN,145.683960,105.02590,NaN,final


## 9. Advanced: SQL joins
Here we inspect the schemas of two HDX Datastore resources (WFP rainfall and Humanitarian Needs Assessment) to identify their available fields, then build and run an SQL query that joins the two datasets for Afghanistan by matching their locations, producing a combined DataFrame with rainfall and affected people in the Food Security sector. This allows us to fetch data from two resources at once.

In [9]:
show_schema(RAINFALL_RESOURCE_ID, "First (Rainfall)")
show_schema(HNO_RESOURCE_ID, "Second (Affected People)")

First (Rainfall) schema (15 fields):
  date: timestamp | is_index=False
  adm_level: numeric | is_index=False
  adm_id: numeric | is_index=False
  PCODE: text | is_index=False
  n_pixels: numeric | is_index=False
  rfh: numeric | is_index=False
  rfh_avg: numeric | is_index=False
  r1h: numeric | is_index=False
  r1h_avg: numeric | is_index=False
  r3h: numeric | is_index=False
  r3h_avg: numeric | is_index=False
  rfq: numeric | is_index=False
  r1q: numeric | is_index=False
  r3q: numeric | is_index=False
  version: text | is_index=False
Second (Affected People) schema (21 fields):
  location_code: text | is_index=False
  has_hrp: text | is_index=False
  in_gho: text | is_index=False
  provider_admin1_name: text | is_index=False
  provider_admin2_name: text | is_index=False
  admin1_code: text | is_index=False
  admin1_name: text | is_index=False
  admin2_code: text | is_index=False
  admin2_name: text | is_index=False
  admin_level: numeric | is_index=False
  sector_code: text | is_

In [10]:
# SQL join HNO and rainfall for Afghanistan 2025
# One row per admin1_code per year with food security affected people and average rainfall
sql = f"""
WITH rain AS (
  SELECT
    "PCODE" AS admin2_code,
    TO_CHAR(date, 'YYYY')  AS year,
    AVG(rfh_avg) AS avg_rainfall
  FROM "{RAINFALL_RESOURCE_ID}"
  WHERE TO_CHAR(date, 'YYYY') = '2025'
  GROUP BY "PCODE", year
),

hno AS (
  SELECT
    admin1_code,
    admin2_code,
    admin1_name,
    admin2_name,
    TO_CHAR(reference_period_end, 'YYYY') AS year,
    population
  FROM "{HNO_RESOURCE_ID}"
  WHERE location_code      = 'AFG'
    AND sector_name        = 'Food Security'
    AND population_status  = 'INN'
    AND lower(category)    = 'total'
)

SELECT
  h.admin1_code,
  h.admin2_code,
  h.admin1_name,
  h.admin2_name,
  h.year,
  h.population       AS affected_population,
  r.avg_rainfall
FROM hno h
JOIN rain r
  ON h.admin2_code = r.admin2_code
 AND h.year       = r.year
ORDER BY h.admin1_code, h.admin2_code
"""

sql_data = datastore_search_sql(sql)
df_joined = pd.DataFrame(sql_data.get("result", {}).get("records", []))

print(f"Joined rows for AFG in 2025: {len(df_joined)}")
df_joined.head()

Joined rows for AFG in 2025: 398


,admin1_code,admin2_code,admin1_name,admin2_name,year,affected_population,avg_rainfall
0,AF01,AF0101,Kabul,Kabul,2025,1285085,8.789394
1,AF01,AF0102,Kabul,Paghman,2025,73730,11.269408
2,AF01,AF0103,Kabul,Chahar Asyab,2025,21374,9.074916
3,AF01,AF0104,Kabul,Bagrami,2025,156742,8.786410
4,AF01,AF0105,Kabul,Deh Sabz,2025,49294,9.028668



## 10. Troubleshooting quick checks

Here are some quick sanity checks to help troubleshoot. If you see empty results or errors:
1. Verify your `"resource_id"` is correct and the resource has HDX Tabular Data endpoints access enabled.
2. Confirm the `"API_TOKEN"` is set in this session and is correct for your account.
3. Remove filters and try `limit=1` again.  
4. For SQL, ensure the table name is quoted `"resource_id"` and the query is URL-encoded by the helper.  
5. Slow or intermittent errors can be rate limiting. Re-run cells after a short pause and add throttling.
   
*For all other questions, please refer to our [documentation](https://un-ocha-centre-for-humanitarian.gitbook.io/hdx-docs/build-with-hdx/build-with-hdx/hdx-api/tabular-data-endpoints).*